# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

设置embedding - zhipu

In [1]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=emb_bgem3
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

C:\Users\will\AppData\Local\Temp\ipykernel_7216\3317253322.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from zhipuai_embedding import ZhipuAIEmbeddings
C:\Users\will\AppData\Local\Temp\ipykernel_7216\3317253322.py:25: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollam

向量数据库已成功加载。


In [2]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：891


In [3]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：891


In [4]:
question = "安全加固的流程?"
docs = vectordb.similarity_search(question,k=15)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：15


打印一下检索到的内容

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [ ]:
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="glm-4",
#     openai_api_key=zhipuai_api_key,
#     openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
# )

In [ ]:
# from langchain_community.llms import Ollama
# llm_deepseek = Ollama(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)

In [5]:
from langchain_deepseek import ChatDeepSeek
_ = load_dotenv(find_dotenv())    # read local .env file
deepseek_api_key = os.environ['DEEPSEEK_API_KEY']
llm_deepseek = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    # max_retries=2,
    api_key=deepseek_api_key,
    # other params...
)

In [ ]:
# from langchain_deepseek import ChatDeepSeek

# _ = load_dotenv(find_dotenv())    # read local .env file
# deepseek_api_key = os.environ['DEEPSEEK_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="deepseek-chat",
#     openai_api_key=deepseek_api_key,
#     openai_api_base="https://api.deepseek.com"
# )

## 3. 构建检索问答链

prompts

In [6]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [7]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(llm_deepseek,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [8]:
question_1 = "什么是安全加固？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

C:\Users\will\AppData\Local\Temp\ipykernel_7216\1351069532.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question_1})


大模型+知识库后回答 question_1 的结果：
安全加固是指通过修改系统配置、更新补丁、优化权限等措施，提升系统的安全性和防御能力，减少漏洞风险。例如修改文件描述符限制、漏洞扫描等操作都属于安全加固范畴。谢谢你的提问！


In [9]:
question_1 = "严威是谁？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
我不知道。谢谢你的提问！


In [10]:
question_1 = "VMAX是什么？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
ZXVMAX-S（ValueMAX）是面向网络运维和运营的多维价值分析系统，支持从网元、用户、终端、业务等多维度进行数据挖掘和分析，涵盖实时和事后分析模式。它主要用于运营商网络建设、运维优化及客户服务等场景。谢谢你的提问！


### 创建一个基于模板的检索链： rerank检索版本

In [ ]:
# # 测试API

# import cohere
# # co = cohere.ClientV2()
# co = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# docs = [
#     "Carson City is the capital city of the American state of Nevada.",
#     "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
#     "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
#     "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
#     "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
# ]
# response = co.rerank(
#     model="rerank-v3.5",
#     query="What is the capital of the United States?",
#     documents=docs,
#     top_n=3,
# )
# print(response)



In [11]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)


C:\Users\will\AppData\Local\Temp\ipykernel_7216\2460964250.py:9: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(


### 创建一个基于模板的检索链： BM25检索版本

## 4.检索问答链效果测试

In [12]:
question_1 = "VMAX是什么？"
question_2 = "严威是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [13]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
VMAX（ZXVMAX-S）是面向网络运维和运营的端到端多维价值分析系统，主要用于语音业务的质量评估、问题定位及优化，支持实时分析和事后分析两种模式。它提供用户分析、网络分析、专题分析和评估优化四大功能，帮助运营商提升网络运维效率和客户服务质量。谢谢你的提问！


In [14]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [15]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm_deepseek.predict(prompt_template)

C:\Users\will\AppData\Local\Temp\ipykernel_7216\4056581889.py:5: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_deepseek.predict(prompt_template)


'VMAX是戴尔科技集团（Dell Technologies）旗下EMC公司推出的一款**高端企业级存储阵列**系列，专为关键业务和大规模数据中心设计，提供高性能、高可靠性和可扩展性。以下是其核心特点：\n\n### 1. **架构与技术**\n   - **基于SAN（存储区域网络）**：支持块级存储，适用于需要低延迟和高吞吐量的场景。\n   - **横向扩展架构**：通过动态添加引擎（节点）和存储容量实现线性扩展，满足不断增长的业务需求。\n   - **多控制器设计**：采用冗余控制器确保高可用性，避免单点故障。\n\n### 2. **关键特性**\n   - **高性能**：利用高速缓存（如闪存加速）、多核处理器和并行I/O处理技术，适合OLTP、虚拟化等密集型工作负载。\n   - **数据保护**：内置快照、克隆、远程复制（如SRDF）等功能，支持灾难恢复。\n   - **混合存储支持**：可整合闪存（SSD）和传统硬盘（HDD），优化成本与性能平衡。\n\n### 3. **应用场景**\n   - 关键数据库（如Oracle、SAP）\n   - 大规模虚拟化环境（VMware、Hyper-V）\n   - 云计算和私有云基础设施\n\n### 4. **版本演进**\n   - **Symmetrix VMAX**：早期版本，基于传统存储架构。\n   - **VMAX3**（2014年推出）：引入嵌入式管理（HYPERMAX OS）和全闪存配置选项。\n   - **VMAX All Flash**：专为全闪存环境优化，提供微秒级延迟。\n   - 后续整合为**PowerMax**（2018年发布）：结合NVMe和SCM（存储级内存），支持AI驱动的自动化管理。\n\n### 5. **现状**\n   VMAX系列已逐步被新一代**PowerMax**取代，后者在性能、智能化和NVMe支持方面进一步升级，但VMAX仍在许多企业中长期服役。\n\n如需更详细的技术参数或与竞品（如HPE XP、IBM DS8000）的对比，可进一步补充说明。'

In [16]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm_deepseek.predict(prompt_template)

'关于“严威”这一名字，目前没有广泛知名或公开可查的权威信息。可能的情况包括：\n\n1. **非公众人物**  \n   该名字可能属于普通人，未在公共领域（如新闻、学术、社交平台等）留下显著记录。\n\n2. **特定领域的从业者**  \n   可能是某个专业领域（如企业、地方机构、学术圈等）的成员，但缺乏公开资料。\n\n3. **笔名/别名**  \n   少数情况下可能是创作者或网络用户的化名，需结合具体上下文判断。\n\n**建议**：  \n- 提供更多背景信息（如行业、地区、成就等）以便缩小范围。  \n- 检查名字是否有拼写变体（如严巍、闫威等）。  \n\n如需进一步帮助，请补充说明。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能-v2025

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [17]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_7216\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

v2024

In [19]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm_deepseek,
    retriever=retriever,
    memory=memory
)
question = "什么是安全加固？"
result = qa({"question": question})
print(result['answer'])

根据提供的上下文内容，没有明确提及“安全加固”的具体定义或相关说明。文档中虽然包含“安全标准”章节（10.2节），但具体内容未展示，因此无法确定是否涉及安全加固的内容。

安全加固通常指通过技术手段（如配置优化、漏洞修复、权限管控等）提升系统或网络的安全性，但这一术语在您提供的文档片段中未被解释。建议查阅完整文档的“安全标准”部分或相关章节以获取准确信息。

若需进一步帮助，请提供更多上下文。


V2025

不带memory

In [21]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# compressor = CohereRerank(
#     client=cohere_client,
#     top_n=5  # 保留Top5相关文档
# )
compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
#                                        retriever=base_retriever,
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])


大模型+知识库后回答 question_1 的结果：
VMAX-S（ValueMAX）是面向网络运维和运营的端到端多维价值分析系统，主要用于语音业务的质量评估、问题定位及优化，支持实时分析和事后分析两种模式。它提供用户分析、网络分析、专题分析和评估优化四大功能，帮助运营商提升网络运维效率和客户服务质量。谢谢你的提问！


新增MEMORY

In [24]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_deepseek,
    retriever=compression_retriever,
    memory=memory,
    # return_source_documents=True,
    output_key="answer",  # 明确指定存储到内存的键
    combine_docs_chain_kwargs={  # 替代chain_type_kwargs
        "prompt": QA_CHAIN_PROMPT
    },
    verbose=True, # 独立传递verbose参数
)


questions = [
    "什么是VMAX的安全加固？", 
    "安全加固的操作步骤？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 什么是安全加固？
Assistant: 根据提供的上下文内容，没有明确提及“安全加固”的具体定义或相关说明。文档中虽然包含“安全标准”章节（10.2节），但具体内容未展示，因此无法确定是否涉及安全加固的内容。

安全加固通常指通过技术手段（如配置优化、漏洞修复、权限管控等）提升系统或网络的安全性，但这一术语在您提供的文档片段中未被解释。建议查阅完整文档的“安全标准”部分或相关章节以获取准确信息。

若需进一步帮助，请提供更多上下文。
Human: 什么是VMAX的安全加固？
Assistant: 根据上下文，**VMAX的安全加固**主要体现在其可靠性设计和漏洞防护机制中，具体包括以下技术措施：

1. **漏洞扫描合规性**  
   - 系统需满足主流安全漏洞扫描工具（如Nessus和CD工具）的要求，确保设备不存在已知安全漏洞，防止外部攻击（7.2节）。

2. **容错与自愈能力**  
   - 通过关键软件资源的定时检测、数据校验、操作日志保存等手段，减少软件故障对系统的影响，提升系统在错误情况下的自愈能力（7.2节）。

3. **故障自动处理**  
   - 自动检测软硬件故障，并对故障硬件实施隔离、倒换、重启等操作，降低安全风险（7.2节）。

4. **松耦合软件设计**  
   - 采用分层模块化结构，松散耦合的机制限制单个模块故障的扩散范围，增强系统整体稳定性（7.2节）。

5. **数据可靠性保障**  
   - 通过存储保护、校验机制等确保数据完整性，防止数据损坏或篡改（7.3节）。

**总结**：VMAX的安全加固是通过漏洞扫描合规性、容错设计、自动化故障处理及数据保护等综合措施，提升系统抗攻击能力和稳定性。具体实施需结合运营商的安全标准（如漏洞扫描工具的具体要求）。
Foll

## 5. 添加历史对话的记忆功能 - v2024

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "什么是南瓜书？"
result = qa({"question": question})
print(result['answer'])

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
question = "南瓜书包含哪些内容？"
result = qa({"question": question})
print(result['answer'])

可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。